<a href="https://colab.research.google.com/github/stanwisniew/SoftwareRegisterApp/blob/main/QuestionAndAnswerforPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import fitz
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import torch

# This will prompt you for your Hugging Face token
login()




In [10]:
# Extract text from PDF and split it into chunks
def extract_and_chunk_pdf(pdf_path, chunk_size=512):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()

    # Split text into chunks of chunk_size tokens
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks

In [30]:
# Example usage
pdf_path = "/content/Fiat Car Configurator.pdf"
chunks = extract_and_chunk_pdf(pdf_path, chunk_size=512)
print(chunks[:500])  # Print the first 500 characters for a preview

# Load pre-trained Llama 2 tokenizer and model using AutoModel
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

print(f"Number of chunks: {len(chunks)}")



['600 1.2 100HP HYBRID 6 SPEED AUTO\n600 1.2 100HP HYBRID 6 SPEED AUTO\nTHE FIAT YOU HAVE CHOSEN\nRetail price\n£ 23,975\nTotal price\n£ 23,975\nIn case of FCA Bank financing, refer to the following pages \nfor further details.\nYour Fiat Code is\n5F3DEC00\nThe Fiat code and QR Code identify options chosen \nby you.* Take these codes to your dealer so that \nyour configuration can be replicated automatically \nand you can ask for an estimate for your car.\n*After 45 days, the codes may no longer be valid\nVehicles in stock\nD', 'ealer locator\nTest drive\n600 1.2 100HP HYBRID 6 SPEED AUTO\n14-11-24 | FIAT CODE 5F3DEC00\n2\nCOLOURS AND WHEELS\nRed\nSTANDARD\n17" Alloy Wheels With Diamond-Cut Finishing\n£ 0\n600 1.2 100HP HYBRID 6 SPEED AUTO\n14-11-24 | FIAT CODE 5F3DEC00\n3\nSTANDARD EQUIPMENT\nAUDIO / TELEMATICS\n4 Speakers Audio System\nUconnect™ Infotainment Touchscreen 10\'\' with Wireless Apple Carplay / \nAndroid Auto\nCOMFORT/FUNCTIONALITY\n7\'\' TFT Digital Cluster\nManual Air C

In [24]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Add padding token to the tokenizer if not already present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token if pad_token is not set

In [33]:
def ask_question(question, chunks):
    # Take the first 5 chunks as context, or adjust this number as needed
    context = " ".join(chunks[:50])  # Adjust based on your chunk size

    # Format the prompt clearly for question answering
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"

    # Now, generate the model's response
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    # Generate the output from the model
    output = model.generate(input_ids, max_new_tokens=150)

    # Decode the generated response (skip special tokens)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract the answer part (it should be after "Answer:")
    answer = generated_text.split("Answer:")[1].strip()

    return answer



In [34]:
# Example of asking a question
question = "What is the price of the car?"
response = ask_question(question, chunks)
print(response)

The retail price of the car is £ 23,975.
